In [3]:
# needed libraries
import os
import pandas as pd
import numpy as np
import boto3
import sagemaker
import awswrangler as wr
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
import joblib
import warnings

warnings.filterwarnings("ignore")

In [4]:
sm_boto3 = boto3.client("sagemaker")
sm_session = sagemaker.Session()

In [5]:
region = sm_session.boto_session.region_name

In [6]:
s3_client = boto3.client("s3", region_name=region)

In [7]:
s3_resource = boto3.resource("s3")

In [8]:
s3_bucket = sm_session.default_bucket()

In [9]:
"""
Note: if you are not running this notebook from SageMaker Studio or SageMaker Classic Notebooks you will need to instanatiate 
the sagemaker_execution_role_name with an AWS role that has SageMakerFullAccess and SageMakerFeatureStoreFullAccess
"""
sagemaker_execution_role_name = "new_role"
try:
    sagemaker_role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client("iam")
    sagemaker_role = iam.get_role(RoleName=sagemaker_execution_role_name)["Role"]["Arn"]
    print(f"\n instantiating sagemaker_role with supplied role name : {sagemaker_role}")

account_id = boto3.client("sts").get_caller_identity()["Account"]

[05/06/25 14:42:52] WARNING  Couldn't call 'get_role' to get Role ARN from role name                ]8;id=430618;file:///Users/mac/Library/Caches/pypoetry/virtualenvs/fraud_detection-Hmgn82TP-py3.13/lib/python3.13/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=787571;file:///Users/mac/Library/Caches/pypoetry/virtualenvs/fraud_detection-Hmgn82TP-py3.13/lib/python3.13/site-packages/sagemaker/session.py#6101\6101]8;;\
                             emmanuelnenchin@gmail.com to get Role path.                                           


 instantiating sagemaker_role with supplied role name : arn:aws:iam::593793065896:role/new_role


In [10]:
print(f"Region name {region}")
print(f"Using bucket {s3_bucket}")

Region name us-east-1
Using bucket sagemaker-us-east-1-593793065896


In [11]:
df = pd.read_csv(r"../data/paysim.csv")
print(df.shape)
df.head()

(6362620, 11)


,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6362620 entries, 0 to 6362619
Data columns (total 11 columns):
 #   Column          Dtype  
---  ------          -----  
 0   step            int64  
 1   type            object 
 2   amount          float64
 3   nameOrig        object 
 4   oldbalanceOrg   float64
 5   newbalanceOrig  float64
 6   nameDest        object 
 7   oldbalanceDest  float64
 8   newbalanceDest  float64
 9   isFraud         int64  
 10  isFlaggedFraud  int64  
dtypes: float64(5), int64(3), object(3)
memory usage: 534.0+ MB


In [13]:
df.count()

step              6362620
type              6362620
amount            6362620
nameOrig          6362620
oldbalanceOrg     6362620
newbalanceOrig    6362620
nameDest          6362620
oldbalanceDest    6362620
newbalanceDest    6362620
isFraud           6362620
isFlaggedFraud    6362620
dtype: int64

In [14]:
df.duplicated().sum()

0

In [15]:
df.nunique()

step                  743
type                    5
amount            5316900
nameOrig          6353307
oldbalanceOrg     1845844
newbalanceOrig    2682586
nameDest          2722362
oldbalanceDest    3614697
newbalanceDest    3555499
isFraud                 2
isFlaggedFraud          2
dtype: int64

In [16]:
df.describe()

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
count,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06
mean,2.433972e+02,1.798619e+05,8.338831e+05,8.551137e+05,1.100702e+06,1.224996e+06,1.290820e-03,2.514687e-06
std,1.423320e+02,6.038582e+05,2.888243e+06,2.924049e+06,3.399180e+06,3.674129e+06,3.590480e-02,1.585775e-03
min,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.560000e+02,1.338957e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,2.390000e+02,7.487194e+04,1.420800e+04,0.000000e+00,1.327057e+05,2.146614e+05,0.000000e+00,0.000000e+00
75%,3.350000e+02,2.087215e+05,1.073152e+05,1.442584e+05,9.430367e+05,1.111909e+06,0.000000e+00,0.000000e+00
max,7.430000e+02,9.244552e+07,5.958504e+07,4.958504e+07,3.560159e+08,3.561793e+08,1.000000e+00,1.000000e+00


In [17]:
clean_df = df.drop(columns=["nameOrig", "nameDest"])
clean_df.head()

,step,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,170136.0,160296.36,0.0,0.0,0,0
1,1,PAYMENT,1864.28,21249.0,19384.72,0.0,0.0,0,0
2,1,TRANSFER,181.00,181.0,0.00,0.0,0.0,1,0
3,1,CASH_OUT,181.00,181.0,0.00,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,41554.0,29885.86,0.0,0.0,0,0


In [65]:
#for feature in clean_df.columns:
#    fig = px.histogram(data_frame=clean_df, x=feature, color="isFraud",
#                       barmode='group',
#                       title=f'Distribution of {feature}', nbins=5,
#                       text_auto=True)
#    fig.update_yaxes(type='log', title='Logarithmic Count')
#    fig.show()

In [19]:
def outlier_removal_iqr(data, column):
    Q1 = data[column].quantile(0.30)
    Q3 = data[column].quantile(0.70)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return data[(data[column] >= lower_bound) & (data[column] <= upper_bound)]
def outlier_removal_zscore(data, column):
    mean = data[column].mean()
    std_dev = data[column].std()
    z_score_threshold = 3
    lower_bound = mean - z_score_threshold * std_dev
    upper_bound = mean + z_score_threshold * std_dev
    return data[(data[column] >= lower_bound) & (data[column] <= upper_bound)]
def outlier_removal(data, column):
    data = outlier_removal_iqr(data, column)
    #data = outlier_removal_zscore(data, column)
    return data


In [20]:

cleaned_df = pd.DataFrame()

for feature in clean_df.columns.drop(["isFraud", "isFlaggedFraud", "type"]):

    cleaned_feature_data = outlier_removal(clean_df, feature)

    cleaned_data = pd.concat([cleaned_df, cleaned_feature_data], axis=0)

cleaned_df = cleaned_data.drop_duplicates()


cleaned_df = cleaned_data.reset_index(drop=True)

print(cleaned_df.shape)
cleaned_df.head()

(5377272, 9)


,step,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,170136.0,160296.36,0.0,0.0,0,0
1,1,PAYMENT,1864.28,21249.0,19384.72,0.0,0.0,0,0
2,1,TRANSFER,181.00,181.0,0.00,0.0,0.0,1,0
3,1,CASH_OUT,181.00,181.0,0.00,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,41554.0,29885.86,0.0,0.0,0,0


In [21]:
cleaned_df.isnull().sum()

step              0
type              0
amount            0
oldbalanceOrg     0
newbalanceOrig    0
oldbalanceDest    0
newbalanceDest    0
isFraud           0
isFlaggedFraud    0
dtype: int64

In [22]:
cleaned_df.describe()

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
count,5.377272e+06,5.377272e+06,5.377272e+06,5.377272e+06,5.377272e+06,5.377272e+06,5.377272e+06,5.377272e+06
mean,2.432560e+02,1.195135e+05,7.614295e+05,7.818377e+05,2.997858e+05,3.544446e+05,1.268301e-03,2.975486e-06
std,1.423943e+02,1.828263e+05,2.757611e+06,2.790687e+06,4.715125e+05,5.104175e+05,3.559063e-02,1.724957e-03
min,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.560000e+02,1.088576e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,2.380000e+02,4.992126e+04,1.581900e+04,0.000000e+00,0.000000e+00,7.291177e+04,0.000000e+00,0.000000e+00
75%,3.340000e+02,1.768874e+05,1.051510e+05,1.407469e+05,4.423788e+05,5.512233e+05,0.000000e+00,0.000000e+00
max,7.430000e+02,1.000000e+07,5.958504e+07,4.958504e+07,1.306083e+07,2.059199e+06,1.000000e+00,1.000000e+00


In [23]:
correlation_matrix = cleaned_df.corr(numeric_only=True)

In [66]:
#plt.figure(figsize=(12, 8))
#sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.3f')
#plt.title('Correlation Matrix')
#plt.show()

In [25]:
cleaned_df = cleaned_df.drop(columns=["newbalanceOrig", "oldbalanceDest", "isFlaggedFraud"])
cleaned_df.head()

,step,type,amount,oldbalanceOrg,newbalanceDest,isFraud
0,1,PAYMENT,9839.64,170136.0,0.0,0
1,1,PAYMENT,1864.28,21249.0,0.0,0
2,1,TRANSFER,181.00,181.0,0.0,1
3,1,CASH_OUT,181.00,181.0,0.0,1
4,1,PAYMENT,11668.14,41554.0,0.0,0


In [26]:
cleaned_df = cleaned_df[~((cleaned_df["oldbalanceOrg"] > 1) & (cleaned_df["newbalanceDest"] < 1))]
cleaned_df.reset_index(drop=True, inplace=True)

In [27]:
cleaned_df.head()

,step,type,amount,oldbalanceOrg,newbalanceDest,isFraud
0,1,DEBIT,5337.77,41720.0,40348.79,0
1,1,DEBIT,9644.94,4465.0,157982.12,0
2,1,CASH_OUT,229133.94,15325.0,51513.44,0
3,1,DEBIT,9302.79,11299.0,16896.70,0
4,1,PAYMENT,9920.52,0.0,0.00,0


In [28]:
cleaned_df.shape

(3716587, 6)

In [29]:
# Probability Ratio Encoding
default_prob = cleaned_df[cleaned_df["isFraud"] == 1]["type"].value_counts() / cleaned_df["type"].value_counts()
cleaned_df["type_prob_ratio_encoded"] = cleaned_df["type"].map(default_prob)

In [30]:
cleaned_df = cleaned_df.drop(columns=["type"])
cleaned_df = cleaned_df.dropna()
cleaned_df.head()

,step,amount,oldbalanceOrg,newbalanceDest,isFraud,type_prob_ratio_encoded
2,1,229133.94,15325.00,51513.44,0,0.001546
12,1,110414.71,26845.41,2415.16,0,0.001546
14,1,56953.90,1942.02,64106.18,0,0.001546
15,1,62610.80,79114.00,8383.29,0,0.000092
16,1,82940.31,3017.87,49864.36,0,0.001546


In [31]:
train_set, test_set = train_test_split(cleaned_df, test_size=0.3, random_state=42, stratify=cleaned_df["isFraud"])

In [32]:
print(train_set.shape)
print(test_set.shape)

(1431629, 6)
(613556, 6)


In [52]:
train_set.to_csv("../data/train_set.csv", index=False)
test_set.to_csv("../data/test_set.csv", index=False)

In [ ]:
import os

train_file_exists = os.path.exists("train_set.csv")
test_file_exists = os.path.exists("test_set.csv")

print(f"Train file saved: {train_file_exists}")
print(f"Test file saved: {test_file_exists}")

Train file saved: True
Test file saved: True


In [56]:
# send data to S3. SageMaker will take training data from s3
sk_prefix = "sagemaker/fraud_det/sklearncontainer"
trainpath = sm_session.upload_data(
    path="train_set.csv", bucket=s3_bucket, key_prefix=sk_prefix
)

testpath = sm_session.upload_data(
    path="test_set.csv", bucket=s3_bucket, key_prefix=sk_prefix
)
print(trainpath)
print(testpath)

s3://sagemaker-us-east-1-593793065896/sagemaker/fraud_det/sklearncontainer/train_set.csv
s3://sagemaker-us-east-1-593793065896/sagemaker/fraud_det/sklearncontainer/test_set.csv


In [57]:
from sagemaker.sklearn.estimator import SKLearn

FRAMEWORK_VERSION = "1.0-1"

sklearn_estimator = SKLearn(
    entry_point="fraud_detection.py",
    role="arn:aws:iam::593793065896:role/new_role",
    instance_count=1,
    instance_type="ml.m5.large",
    framework_version=FRAMEWORK_VERSION,
    base_job_name="FD-custom-sklearn",
    hyperparameters={
        "n_estimators": 100,
        "random_state": 42,
    },
    source_dir=".",
    dependencies=["../requirements.txt"],
    use_spot_instances = True,
    max_wait = 7200,
    max_run = 3600,
    output_path="s3://sagemaker-us-east-1-593793065896/sagemaker/fraud_det/sklearncontainer/sagemaker_model_output"
)


In [58]:
# launch training job, with asynchronous call
sklearn_estimator.fit({"train": trainpath, "test": testpath}, wait=True)

[05/06/25 15:21:53] INFO     SageMaker Python SDK will collect telemetry to help us better  ]8;id=925745;file:///Users/mac/Library/Caches/pypoetry/virtualenvs/fraud_detection-Hmgn82TP-py3.13/lib/python3.13/site-packages/sagemaker/telemetry/telemetry_logging.py\telemetry_logging.py]8;;\:]8;id=961711;file:///Users/mac/Library/Caches/pypoetry/virtualenvs/fraud_detection-Hmgn82TP-py3.13/lib/python3.13/site-packages/sagemaker/telemetry/telemetry_logging.py#91\91]8;;\
                             understand our user's needs, diagnose issues, and deliver                             
                             additional features.                                                                  
                             To opt out of telemetry, please disable via TelemetryOptOut                           
                             parameter in SDK defaults config. For more information, refer                         
                             to                                                                                    
                             https://sagemaker.readthedocs.io/en/stable/overview.html#confi                        
                             guring-and-using-defaults-with-the-sagemaker-python-sdk.                              

[05/06/25 15:22:43] INFO     Creating training-job with name:                                       ]8;id=827195;file:///Users/mac/Library/Caches/pypoetry/virtualenvs/fraud_detection-Hmgn82TP-py3.13/lib/python3.13/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=60325;file:///Users/mac/Library/Caches/pypoetry/virtualenvs/fraud_detection-Hmgn82TP-py3.13/lib/python3.13/site-packages/sagemaker/session.py#1042\1042]8;;\
                             FD-custom-sklearn-2025-05-06-14-21-53-424                                             

2025-05-06 14:22:46 Starting - Starting the training job...
2025-05-06 14:23:23 Downloading - Downloading input data...
2025-05-06 14:23:54 Downloading - Downloading the training image......
2025-05-06 14:24:49 Training - Training image download completed. Training in progress..2025-05-06 14:24:54,067 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2025-05-06 14:24:54,070 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2025-05-06 14:24:54,073 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2025-05-06 14:24:54,090 sagemaker_sklearn_container.training INFO     Invoking user training script.
2025-05-06 14:24:58,257 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
/miniconda3/bin/python -m pip install -r requirements.txt
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 884.9/884.9 kB 46.3 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  P

In [59]:
sklearn_estimator.latest_training_job.wait(logs="None")
artifact = sm_boto3.describe_training_job(
    TrainingJobName=sklearn_estimator.latest_training_job.name
)["ModelArtifacts"]["S3ModelArtifacts"]

print("Model artifact persisted at " + artifact)


2025-05-06 14:32:44 Starting - Preparing the instances for training
2025-05-06 14:32:44 Downloading - Downloading the training image
2025-05-06 14:32:44 Training - Training image download completed. Training in progress.
2025-05-06 14:32:44 Uploading - Uploading generated training model
2025-05-06 14:32:44 Completed - Training job completed
Model artifact persisted at s3://sagemaker-us-east-1-593793065896/sagemaker/fraud_det/sklearncontainer/sagemaker_model_output/FD-custom-sklearn-2025-05-06-14-21-53-424/output/model.tar.gz


In [60]:
artifact

's3://sagemaker-us-east-1-593793065896/sagemaker/fraud_det/sklearncontainer/sagemaker_model_output/FD-custom-sklearn-2025-05-06-14-21-53-424/output/model.tar.gz'

In [61]:
from sagemaker.sklearn.model import SKLearnModel
from time import gmtime, strftime

model_name = "Custom-sklearn-model-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
model = SKLearnModel(
    name =  model_name,
    model_data=artifact,
    role="arn:aws:iam::593793065896:role/new_role",
    entry_point="fraud_detection.py",
    framework_version=FRAMEWORK_VERSION,
)

In [62]:
model_name

'Custom-sklearn-model-2025-05-06-14-33-36'

In [64]:
endpoint_name = "Custom-sklearn-model-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print("EndpointName={}".format(endpoint_name))

predictor = model.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.large",
    endpoint_name=endpoint_name,
)

EndpointName=Custom-sklearn-model-2025-05-06-15-05-43


[05/06/25 16:05:44] INFO     Creating model with name: Custom-sklearn-model-2025-05-06-14-33-36     ]8;id=682136;file:///Users/mac/Library/Caches/pypoetry/virtualenvs/fraud_detection-Hmgn82TP-py3.13/lib/python3.13/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=613984;file:///Users/mac/Library/Caches/pypoetry/virtualenvs/fraud_detection-Hmgn82TP-py3.13/lib/python3.13/site-packages/sagemaker/session.py#4094\4094]8;;\

[05/06/25 16:05:46] WARNING  Using already existing model: Custom-sklearn-model-2025-05-06-14-33-36 ]8;id=904525;file:///Users/mac/Library/Caches/pypoetry/virtualenvs/fraud_detection-Hmgn82TP-py3.13/lib/python3.13/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=98387;file:///Users/mac/Library/Caches/pypoetry/virtualenvs/fraud_detection-Hmgn82TP-py3.13/lib/python3.13/site-packages/sagemaker/session.py#4105\4105]8;;\

                    INFO     Creating endpoint-config with name                                     ]8;id=321396;file:///Users/mac/Library/Caches/pypoetry/virtualenvs/fraud_detection-Hmgn82TP-py3.13/lib/python3.13/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=514662;file:///Users/mac/Library/Caches/pypoetry/virtualenvs/fraud_detection-Hmgn82TP-py3.13/lib/python3.13/site-packages/sagemaker/session.py#6019\6019]8;;\
                             Custom-sklearn-model-2025-05-06-15-05-43                                              

[05/06/25 16:05:47] INFO     Creating endpoint with name Custom-sklearn-model-2025-05-06-15-05-43   ]8;id=524676;file:///Users/mac/Library/Caches/pypoetry/virtualenvs/fraud_detection-Hmgn82TP-py3.13/lib/python3.13/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=599628;file:///Users/mac/Library/Caches/pypoetry/virtualenvs/fraud_detection-Hmgn82TP-py3.13/lib/python3.13/site-packages/sagemaker/session.py#4841\4841]8;;\

-

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:4                                                                                    │
│                                                                                                  │
│   1 endpoint_name = "Custom-sklearn-model-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())            │
│   2 print("EndpointName={}".format(endpoint_name))                                               │
│   3                                                                                              │
│ ❱ 4 predictor = model.deploy(                                                                    │
│   5 │   initial_instance_count=1,                                                                │
│   6 │   instance_type="ml.m5.large",                                                             │
│   7 │   endpoint_name=endpoint_name,                                                             │
│                                                                                                  │
│ /Users/mac/Library/Caches/pypoetry/virtualenvs/fraud_detection-Hmgn82TP-py3.13/lib/python3.13/si │
│ te-packages/sagemaker/model.py:1814 in deploy                                                    │
│                                                                                                  │
│   1811 │   │   │   │   )                                                                         │
│   1812 │   │   │   │   self.sagemaker_session.update_endpoint(self.endpoint_name, endpoint_conf  │
│   1813 │   │   │   else:                                                                         │
│ ❱ 1814 │   │   │   │   self.sagemaker_session.endpoint_from_production_variants(                 │
│   1815 │   │   │   │   │   name=self.endpoint_name,                                              │
│   1816 │   │   │   │   │   production_variants=[production_variant],                             │
│   1817 │   │   │   │   │   tags=tags,                                                            │
│                                                                                                  │
│ /Users/mac/Library/Caches/pypoetry/virtualenvs/fraud_detection-Hmgn82TP-py3.13/lib/python3.13/si │
│ te-packages/sagemaker/session.py:6022 in endpoint_from_production_variants                       │
│                                                                                                  │
│   6019 │   │   logger.info("Creating endpoint-config with name %s", name)                        │
│   6020 │   │   self.sagemaker_client.create_endpoint_config(**config_options)                    │
│   6021 │   │                                                                                     │
│ ❱ 6022 │   │   return self.create_endpoint(                                                      │
│   6023 │   │   │   endpoint_name=name,                                                           │
│   6024 │   │   │   config_name=name,                                                             │
│   6025 │   │   │   tags=endpoint_tags,                                                           │
│                                                                                                  │
│ /Users/mac/Library/Caches/pypoetry/virtualenvs/fraud_detection-Hmgn82TP-py3.13/lib/python3.13/si │
│ te-packages/sagemaker/session.py:4856 in create_endpoint                                         │
│                                                                                                  │
│   4853 │   │   │   │   self.endpoint_arn = res["EndpointArn"]                                    │
│   4854 │   │   │                                                                                 │
│   4855 │   │   │   if wait:                                                                      │
│ ❱ 4856 │   │   │   │   self.wait_for_endpoint(endpoint_name

In [ ]:
endpoint_name

In [ ]:
features = test_set.drop(columns=["isFraud"]).columns

In [ ]:
test_set[features][0:2].values.tolist()

In [ ]:
print(predictor.predict(test_set[features][0:2].values.tolist()))